In [1]:
import numpy as np
from pybullet_environment import *

env = RobotEnvironment(manipulator = False)

pybullet build time: Jul 17 2023 13:38:55


argv[0]=
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=3
argv[0] = --unused
argv[1] = 
argv[2] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce GTX 1660 Ti/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 535.86.05
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 535.86.05
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce GTX 1660 Ti/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


In [2]:
def euler_to_rotation_matrix(yaw, pitch, roll):
        
    Rz = np.array([[np.cos(yaw), -np.sin(yaw), 0],
                [np.sin(yaw), np.cos(yaw), 0],
                [0, 0, 1]])

    Ry = np.array([[np.cos(pitch), 0, np.sin(pitch)],
                [0, 1, 0],
                [-np.sin(pitch), 0, np.cos(pitch)]])

    Rx = np.array([[1, 0, 0],
                [0, np.cos(roll), -np.sin(roll)],
                [0, np.sin(roll), np.cos(roll)]])

    R = Rz @ (Ry @ Rx)
    
    return R

In [3]:
def define_vertices(pos, ori, dims):

    l, b, h = dims[0], dims[1], dims[2]

    local_vertices = np.array([[-l/2, -b/2, -h/2, 1],
                                [ l/2, -b/2, -h/2, 1],
                                [ l/2,  b/2, -h/2, 1],
                                [-l/2,  b/2, -h/2, 1],
                                [-l/2, -b/2,  h/2, 1],
                                [ l/2, -b/2,  h/2, 1],
                                [ l/2,  b/2,  h/2, 1],
                                [-l/2,  b/2,  h/2, 1]])
    
    rot = euler_to_rotation_matrix(ori[0], ori[1], ori[2])

    T = np.eye(4)
    T[:3, :3] = rot
    T[:3, 3] = pos[:]

    global_vertices = (T @ (local_vertices.T)).T

    return global_vertices[:, :-1]  

In [4]:
def find_furthest_vertex(direction, vertices):

    direction = np.reshape(direction, (3, 1))
    local_vertices = vertices - (np.mean(vertices, axis = 0)[np.newaxis, :])

    dot_product = (local_vertices @ direction)[:, 0]
    
    furthest_vertex = vertices[np.argmax(dot_product)]

    return furthest_vertex

def support_mapping(direction, X, Y):

    return find_furthest_vertex(direction, X) - find_furthest_vertex(-direction, Y)
        

In [5]:
def unit_vector(v):

    norm = np.linalg.norm(v)
    return v / norm

def is_equal(tuple1, tuple2):
    
    for arr1, arr2 in zip(tuple1, tuple2):
        if not np.array_equal(arr1, arr2):
            return False
    
    return True

In [6]:
# Debugging Functions:

def has_duplicates(lst):
    flattened_list = [tuple(np.array(element).flatten()) for element in lst]
    return len(flattened_list) != len(set(flattened_list))

def is_point_in_polytope(point, polytope):

    ret = False
    for p in polytope:
        if np.sum(np.abs(point - p)) < 1e-10:
            return p, True
    
    return None, ret

def normal_to_dist(normal, polytope):

    dist_min = np.inf
    for point in polytope:
        dist = np.dot(point, normal)
        if np.abs(dist) < dist_min and dist > 0:
            dist_min = dist
        elif dist == 0:
            print("Origin lies on the polytope face !!")
            break

    return dist_min


In [7]:
def get_face_normal_and_distance(A, B, C):

    BA = A - B
    CA = A - C
    normal = unit_vector(np.cross(CA, BA))
    distance = np.dot(normal, A)
    if distance < 0:
        distance *= -1
        normal = -normal

    return normal.copy(), distance

def get_normals_and_distances_to_faces_of_tetrahedron(tetrahedron):

    faces = []
    normals = []
    dists = []
    min_dist = np.inf

    for i in range(len(tetrahedron)):

        j = (i+1) % len(tetrahedron)
        k = (i+2) % len(tetrahedron)

        A = tetrahedron[i].copy()
        B = tetrahedron[j].copy()
        C = tetrahedron[k].copy()

        normal, dist = get_face_normal_and_distance(A, B, C)
        
        if dist < min_dist:
            min_dist = dist
            min_normal = normal.copy()
            min_index = i

        faces.append((A, B, C))
        normals.append(normal)
        dists.append(dist)

    return min_normal.copy(), min_dist, min_index, faces[:], normals[:], dists[:]

def add_unique_edges(edges, face):

    # My logic behind this function:
    # Unique edges are those that are coupled to only one plane. These edges will have only one entry (a, b)
    # The other edges will have both (a, b) and (b, a), because the edges will be there in opposite order.
    # Therefore, when we try to add (a, b), we see that (b, a) already exists, 
    # so this isn't a unique edge. Then we remove the edge that was already stored as (b, a) since its not a unique edge.
    
    for i in range(3):

        j = (i+1) % 3    

        # Return the edge (b, a) if it is stored in the opposite order from the input (a, b)
        repeated_edge_index = next((k for k in range(len(edges)) if is_equal(edges[k], (face[j], face[i])) or is_equal(edges[k], (face[i], face[j]))), None)
        
        # If a repeated edge was found, remove that edge
        if type(repeated_edge_index) != type(None):
            edges.pop(repeated_edge_index)
        # If no reverse edge was found, append the edge
        else:
            edges.append((face[i], face[j]))

    return edges[:]

def reconstruct_polytope(polytope, normals, faces, dists, next_point):

    unique_edges = []
    i = 0

    while i < len(normals):

        if np.dot(normals[i], next_point) > 0:

            face = faces[i]
            unique_edges = add_unique_edges(unique_edges, face)

            faces.pop(i)
            normals.pop(i)
            dists.pop(i)

            i -= 1

        i += 1
    
    new_faces = []
    for edge in unique_edges:
        new_faces.append((edge[0], edge[1], next_point))

    polytope.append(next_point)

    return polytope, normals[:], faces[:], dists[:], new_faces[:]

def get_normals_and_distances_from_faces(new_faces):

    new_normals = []
    new_dists = []
    min_dist = np.inf

    for i in range(len(new_faces)):
        
        A = new_faces[i][0].copy()
        B = new_faces[i][1].copy()
        C = new_faces[i][2].copy()

        normal, dist = get_face_normal_and_distance(A, B, C)

        if dist < min_dist:
            min_dist = dist
            min_index = i
            new_min_normal = normal.copy()

        new_normals.append(normal)
        new_dists.append(dist)

    return new_min_normal.copy(), min_dist, min_index, new_normals[:], new_dists[:]

In [8]:
def update_simplex(simplex, X, Y):

    # Return 0 means that he origin lies on either a vertex, an edge, or a face

    if len(simplex) == 0:
        direction = np.array([[1., 0., 0.]]).T
        next_point = support_mapping(direction, X, Y)
        simplex.append(next_point)
        return update_simplex(simplex, X, Y)
    
    elif len(simplex) == 1:
        A = simplex[0]
        direction = -A
        if np.linalg.norm(direction) == 0:
            return 0        # Origin lies on a vertex
        next_point = support_mapping(unit_vector(direction), X, Y)
        if np.dot(next_point, direction) < 0:
            return 0        # Origin is outside shape
        simplex.append(next_point)
        return update_simplex(simplex, X, Y)

    elif len(simplex) == 2:
        A, B = simplex[0], simplex[1]
        BA = A - B
        direction = np.cross(np.cross(BA, B), BA)
        if np.linalg.norm(direction) == 0:
            return 0        # Origin lies on an edge
        next_point = support_mapping(unit_vector(direction), X, Y)
        if np.dot(next_point, direction) < 0:
            return 0        # Origin is outside shape
        simplex.append(next_point)
        return update_simplex(simplex, X, Y)

    elif len(simplex) == 3:
        A, B, C = simplex[0], simplex[1], simplex[2]
        BA = A - B
        CA = A - C
        direction = np.cross(CA, BA)
        if np.dot(direction, C) > 0:
            direction = -direction
        if np.linalg.norm(direction) == 0:
            return 0        # Origin lies on a face
        next_point = support_mapping(unit_vector(direction), X, Y)
        if np.dot(next_point, direction) < 0:
            return 0        # Origin is outside shape
        simplex.append(next_point)
        return update_simplex(simplex, X, Y)

    elif len(simplex) == 4:
        
        A, B, C, D = simplex[0], simplex[1], simplex[2], simplex[3]
        
        DA = D - A
        DB = D - B
        DC = D - C
        centroid = (A + B + C + D)/4
        to_centroid = centroid - D
        faces = [(DA, DB), (DB, DC), (DC, DA)]

        direction = None

        for face in faces:
            normal = np.cross(face[0], face[1])
            if np.dot(normal, to_centroid) > 0:
                normal = -normal
            if np.dot(normal, D) < 0:
                direction = normal.copy()

        if type(direction) == type(None):
            return simplex
        
        simplex.pop(0)
        next_point = support_mapping(unit_vector(direction), X, Y)
        if np.dot(next_point, direction) < 0:
            return 0        # Origin is outside shape
        simplex.append(next_point)
        return update_simplex(simplex, X, Y)

In [9]:
def EPA(polytope, X, Y):

    if polytope == 0:
        return 0
    
    min_normal, min_dist, min_index, faces, normals, dists = get_normals_and_distances_to_faces_of_tetrahedron(polytope)

    print("Initial Distances = ", dists)
    
    next_point = support_mapping(min_normal, X, Y)
    new_dist = np.dot(min_normal, next_point)

    if np.abs(new_dist - min_dist) > 0.0001:
        min_dist = np.inf

    while (min_dist == np.inf):

        polytope, normals, faces, dists, new_faces = reconstruct_polytope(polytope, normals, faces, dists, next_point)

        new_min_normal, new_min_dist, new_min_index, new_normals, new_dists = get_normals_and_distances_from_faces(new_faces)

        print("---------------------------------------------------")
        
        old_min_dist = np.inf
        for i in range(len(normals)):
            if dists[i] < old_min_dist:
                old_min_dist = dists[i]
                old_min_normal = normals[i].copy()
                old_min_index = i

        if new_min_dist < old_min_dist:
            min_normal = new_min_normal.copy()
            min_dist = new_min_dist
            min_index = len(normals) + new_min_index
            print("New Min Normal")

        else:
            min_normal = old_min_normal.copy()
            min_dist = old_min_dist
            min_index = old_min_index
            print("Old Min Normal")

        normals.extend(new_normals)
        faces.extend(new_faces)
        dists.extend(new_dists)

        dist_min = normal_to_dist(min_normal, polytope)

        A, B, C = faces[min_index]
        print("Distance to face points = ", (np.dot(A, min_normal), np.dot(B, min_normal), np.dot(C, min_normal)))

        next_point = support_mapping(min_normal, X, Y)
        print("Next Point = ", next_point)
        print("Face = ", faces[min_index])
        new_dist = np.dot(min_normal, next_point)

        print("Is polytope Convex? ", )

        # print(polytope)
        # print(next_point)
        print("Minimum Distance Corresponding to Normal = ", dist_min)
        print(is_point_in_polytope(next_point, polytope))
        print("New Dist, Min Dist = ", new_dist, min_dist)

        # print("Next Point = ", next_point)
        # print("Normal = ", min_normal)
        # print("Distance = ", min_dist)
        # print("Polytope Size = ", len(polytope))
        _ = input("")

        if np.abs(new_dist - min_dist) > 0.0001:
            min_dist = np.inf

    return min_dist


In [10]:
X_pos = np.array([0.2, 0.1, 0.5])
X_ori = np.array([0., 0., 0.])
X_size = np.array([0.5, 0.3, 0.2])

# Slightly colliding:
# Y_pos = np.array([0.56, 0.11, 0.51])
# Y_ori = np.array([0.1, 0.3, -0.4])
# Y_size = np.array([0.15, 0.3, 0.08])

# Fully colliding:
Y_pos = np.array([0.3, 0.11, 0.51])
Y_ori = np.array([0.1, 0.3, -0.4])
Y_size = np.array([0.15, 0.3, 0.08])

# Form the vertices of the shapes:
X = define_vertices(X_pos, X_ori, X_size)
Y = define_vertices(Y_pos, Y_ori, Y_size)
simplex = []

origin_simplex = update_simplex(simplex, X, Y)
print(origin_simplex)

[array([ 0.18104526,  0.26631281, -0.22316478]), array([ 0.24298288, -0.30517837, -0.11155711]), array([0.26153931, 0.02799337, 0.15883674]), array([-0.38104526, -0.28631281,  0.20316478])]


In [105]:
EPA(origin_simplex, X, Y)

Initial Distances =  [0.2340359698317686, 0.18751607284522187, 0.02051765784360747, 0.055339889652950724]
---------------------------------------------------
New Min Normal
Distance to face points =  (0.0553398896529507, 0.05533988965295074, 0.05533988965295074)
Next Point =  [-0.46153931 -0.04799337 -0.17883674]
Face =  (array([ 0.18104526,  0.26631281, -0.22316478]), array([ 0.24298288, -0.30517837, -0.11155711]), array([-0.44298288,  0.28517837,  0.09155711]))
Minimum Distance Corresponding to Normal =  0.0553398896529507
(None, False)
New Dist, Min Dist =  0.37108572328027495 0.0553398896529507
---------------------------------------------------
Old Min Normal
Distance to face points =  (0.15474456527158112, 0.1547445652715811, 0.15474456527158115)
Next Point =  [-0.38104526  0.01368719  0.20316478]
Face =  (array([0.26153931, 0.02799337, 0.15883674]), array([-0.38104526, -0.28631281,  0.20316478]), array([-0.44298288,  0.28517837,  0.09155711]))
Minimum Distance Corresponding to N

KeyboardInterrupt: Interrupted by user

In [14]:
from itertools import combinations

for comb in combinations(origin_simplex, 2):
    env.client_id.addUserDebugLine(comb[0], comb[1], env.colors['purple'], lineWidth = 4)

: 

In [59]:
env.client_id.removeBody(A_id)
env.client_id.removeBody(B_id)

X_pos = np.array([0.2, 0.1, 0.5])
X_ori = np.array([0., 0., 0.])
X_size = np.array([0.5, 0.3, 0.2])

# Slightly colliding:
Y_pos = np.array([0.56, 0.11, 0.51])
Y_ori = np.array([0.1, 0.3, -0.4])
Y_size = np.array([0.15, 0.3, 0.08])

# Fully colliding:
# Y_pos = np.array([0.3, 0.11, 0.51])
# Y_ori = np.array([0.1, 0.3, -0.4])
# Y_size = np.array([0.15, 0.3, 0.08])

vuid = env.client_id.createVisualShape(p.GEOM_BOX, 
                                       halfExtents = X_size/2,
                                       rgbaColor = np.hstack([env.colors['yellow'], np.array([1.0])]))

A_id = env.client_id.createMultiBody(baseVisualShapeIndex = vuid, 
                                       basePosition = X_pos, 
                                       baseOrientation = env.client_id.getQuaternionFromEuler(X_ori))

vuid = env.client_id.createVisualShape(p.GEOM_BOX, 
                                       halfExtents = Y_size/2,
                                       rgbaColor = np.hstack([env.colors['red'], np.array([1.0])]))

B_id = env.client_id.createMultiBody(baseVisualShapeIndex = vuid, 
                                       basePosition = Y_pos, 
                                       baseOrientation = env.client_id.getQuaternionFromEuler(Y_ori))

# Form the vertices of the shapes:
X = define_vertices(X_pos, X_ori, X_size)
Y = define_vertices(Y_pos, Y_ori, Y_size)
simplex = []

out = update_simplex(simplex, X, Y)
print(out)

[array([0.00153931, 0.02799337, 0.15883674]), array([-0.01701712, -0.30517837, -0.11155711]), array([ 0.00153931,  0.02799337, -0.04116326]), array([ 0.00153931, -0.27200663, -0.04116326])]


In [106]:
env.client_id.removeBody(A_id)
env.client_id.removeBody(B_id)

: 

In [44]:
# Form the vertices of the shapes:
X = define_vertices(X_pos, X_ori, X_size)
Y = define_vertices(Y_pos, Y_ori, Y_size)

# Pick a random vertex on the Minkowski Difference:
direction = np.array([[1., 0., 0.]]).T
A = support_mapping(direction, X, Y)
# Remember the support mapping gives us the furthest vertex in a direction from the centroid of the minkowski difference,
# in terms of the Original Origin (that is, global coordinates, and NOT local to the Minkowski Difference Shape)

# Pick second vertex towards origin:
direction = -A
if np.linalg.norm(direction) == 0:
    print(True)
B = support_mapping(unit_vector(direction), X, Y)
if np.dot(B, direction) < 0:
    print(False)

# Pick third vertex towards origin:
BA = A - B
direction = np.cross(np.cross(BA, B), BA)
if np.linalg.norm(direction) == 0:
    print(True)
C = support_mapping(unit_vector(direction), X, Y)
if np.dot(C, direction) < 0:
    print(False)

# Pick fourth vertex towards origin:
CA = A - C
direction = np.cross(CA, BA)
if np.dot(direction, C) > 0:
    direction = -direction
if np.linalg.norm(direction) == 0:
    print(True)
D = support_mapping(unit_vector(direction), X, Y)
if np.dot(D, direction) < 0:
    print(False)

# In the above 3 snippets, if the origin lies on either a vertex, an edge, or a face, the direction will be zero.

# Pick next vertex from tetrahedron:

contains_origin = False
direction = None

DA = D - A
DB = D - B
DC = D - C
centroid = (A + B + C + D)/4
to_centroid = centroid - D
# Face 1:
normal = np.cross(DA, DB)
if np.dot(normal, to_centroid) > 0:
    normal = -normal
if np.dot(normal, D) < 0:
    direction = normal.copy()
# Face 2:
normal = np.cross(DB, DC)
if np.dot(normal, to_centroid) > 0:
    normal = -normal
if np.dot(normal, D) < 0:
    direction = normal.copy()
# Face 3:
normal = np.cross(DC, DA)
if np.dot(normal, to_centroid) > 0:
    normal = -normal
if np.dot(normal, D) < 0:
    direction = normal.copy()

if type(direction) == type(None):
    contains_origin = True


<>:43: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<>:43: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
/tmp/ipykernel_6924/821103382.py:43: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  (DA, DB) (DB, DC) (DC, DA)
/tmp/ipykernel_6924/821103382.py:43: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  (DA, DB) (DB, DC) (DC, DA)
/tmp/ipykernel_6924/821103382.py:43: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  (DA, DB) (DB, DC) (DC, DA)
/tmp/ipykernel_6924/821103382.py:43: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  (DA, DB) (DB, DC) (DC, DA)


NameError: name 'DA' is not defined

In [43]:
np.cross(np.cross([1, 2, 3], [1, 2, 3]), [1, 2, 3])

array([0, 0, 0])

In [32]:


print(support_mapping([1., -0.1, -0.5], X, Y))

[ 0.33298288 -0.30517837 -0.11155711]


In [46]:
l = [1, 2, 3]
l.pop(0)
print(l)

[2, 3]
